In [73]:
import torch as t
from mcs import MCS
from PIL import Image
from scipy import ndimage
import numpy as np
import random

In [ ]:
class STESpike(t.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return (input > 0).float()

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output


In [16]:
grid = t.zeros((2,8,8))
grid[0,4,4] = 1
grid[0,5,5] = 1

grid[1,1,1] = 1
grid[1,1,2] = 1
grid

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]])

In [17]:
(grid == 1).nonzero()

tensor([[0, 4, 4],
        [0, 5, 5],
        [1, 1, 1],
        [1, 1, 2]])

In [71]:
MOORE_OFFSETS = t.tensor([(1, 1), (1, -1), (1, 0), (-1, 0), (-1, 1), (-1, -1), (0, 1), (0, -1)])

def get_moore_nbh(batch:t.tensor):
    batch_size, batch_height, _ = batch.shape
    cell_pixel_coords = (batch == 1).nonzero()
    moore_nbhs = []
    for batch_idx in range(batch_size):
        batch_slice = cell_pixel_coords[:, 0] == batch_idx
        #print(cell_pixel_coords[batch_slice])
        coords = cell_pixel_coords[batch_slice]
        batch_moore_nbhs = []
        for coord in coords:
            nbh_coords = coord[1:]+MOORE_OFFSETS
            idx_pad = t.zeros(8,1) + batch_idx
            nbh_coords = t.hstack((idx_pad, nbh_coords))
            #print(nbh_coords)
            batch_moore_nbhs.append(nbh_coords)
        res = t.vstack(batch_moore_nbhs).unique(dim=0)
        res[res == -1] = 1
        res[res == batch_height] = batch_height - 2
        moore_nbhs.append(res)
    return moore_nbhs


In [81]:
test = get_moore_nbh(grid)[0]
random.choice(test)

tensor([0., 5., 6.])

In [112]:
def MCS(batch, target_vol, temperature):
    batch_size, batch_height, batch_width = batch.shape
    frontiers = get_moore_nbh(batch)
    for batch_idx, frontier in enumerate(frontiers):
        src_coords = random.choice(frontier).type(t.long)
        step_size = random.choice(MOORE_OFFSETS)
        tgt_coords = src_coords.clone()
        tgt_coords[1:] += step_size
        tgt_coords[tgt_coords == -1] = 1
        tgt_coords[tgt_coords == batch_height] = batch_height -2
        print(f"coordinates of the source pixel on the grid: {src_coords}")
        print(f"step size: {step_size}")
        print(f"coordinates of the target pixel on the grid: {tgt_coords}")
        src_i, src_j, src_k = src_coords
        tgt_i, tgt_j, tgt_k = tgt_coords
        
        cur_vol = t.sum(batch[batch_idx])
        vol_change = (-1 * batch[tgt_i, tgt_j, tgt_k]) + batch[src_i, src_j, src_k]
        print("vol change", vol_change)
        adjusted_vol = cur_vol + vol_change
        print(f"adjusted vol: {adjusted_vol}")
        if batch[tgt_i, tgt_j, tgt_k] == batch[src_i, src_j, src_k]:
            # source is equal to target, no update
            print("source is equal to target, no update")
            pass
        elif adjusted_vol > 2 or adjusted_vol <= 0:
            # Changes would violate the hard constraints, no update
            print("Changes would violate the hard constraints, no update")
            pass
        elif cur_vol == 2 and vol_change == -1:
            # Negative Hamiltonian, accepted
            print("Negative Hamiltonian, accepted")
            batch[tgt_i, tgt_j, tgt_k] += vol_change
        else:
            update_probability = t.exp(-((target_vol - adjusted_vol) ** 2) / temperature)
            print(f"update probability: {update_probability}")
            
            residual = t.rand(1)
            
            upd_val = STESpike.apply(update_probability - residual) * vol_change

            batch[tgt_i, tgt_j, tgt_k] += upd_val
    return batch

In [114]:
MCS(grid, 1, 1)

coordinates of the source pixel on the grid: tensor([0, 4, 6])
step size: tensor([ 1, -1])
coordinates of the target pixel on the grid: tensor([0, 5, 5])
vol change tensor(-1.)
adjusted vol: 1.0
Negative Hamiltonian, accepted
coordinates of the source pixel on the grid: tensor([1, 2, 1])
step size: tensor([-1,  1])
coordinates of the target pixel on the grid: tensor([1, 1, 2])
vol change tensor(-1.)
adjusted vol: 1.0
Negative Hamiltonian, accepted


tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]])